In [ ]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [ ]:
from sportwarz import load_counties_geojson

counties_geojson = load_counties_geojson()

In [ ]:
from sportwarz import load_counties_data

co_data_frame = load_counties_data(counties_geojson)

In [ ]:
from sportwarz import load_leagues

league_name = "MLS"

leagues = {
    league_name: {  },
}    

load_leagues(leagues)

In [ ]:
from sportwarz import calculate_distances

calculate_distances(leagues, co_data_frame) 

In [ ]:
from sportwarz import compute_shares
        
compute_shares(leagues, co_data_frame)  

In [ ]:
from sportwarz import compute_output_dataframes

compute_output_dataframes(leagues, co_data_frame)       

MLS as of now

In [ ]:
from sportwarz import render_map, reset_county_styles, heatmap_counties

reset_county_styles(counties_geojson)
heatmap_counties(leagues, counties_geojson, league_name) 

render_map(leagues, league_name, counties_geojson, co_data_frame)

MLS if eight of the larger USL-Championship markets joined.

In [ ]:
from sportwarz import Team, Coordinates, reset_county_styles, add_team, load_counties_geojson, heatmap_counties, copy_with_just_league

team_SAC = Team(
    name = "Sacramento Republic FC",
    venue = "HeartHealth Park",
    L = 1.0,
    S = 4.0,
    coordinates = Coordinates(lat = 38.5805, lon = -121.4964),
    color = "#8B0000",
    state = "California"
)

team_IND = Team(
    name = "Indy Eleven",
    venue = "Eleven Park",
    L = 1.0,
    S = 4.0,
    coordinates = Coordinates(lat = 39.771146, lon = -86.177880),
    color = "#000060",
    state = "Indiana"
)

team_SA = Team(
    name = "San Antonio FC",
    venue = "Toyota Field",
    L = 1.0,
    S = 4.0,
    coordinates = Coordinates(lat = 29.4705, lon = -98.5144),
    color = "#000000",
    state = "Texas"
)

team_DET = Team(
    name = "Detroit City FC",
    L = 1.0,
    S = 4.0,
    venue = "Keyworth Stadium",
    coordinates = Coordinates(lat = 42.4135, lon = -83.0304),
    color = "#A6192E",
    state = "Michigan"
)

team_PHO = Team(
    name = "Phoenix Rising FC",
    venue = "Phoenix Rising Soccer Stadium",
    L = 1.0,
    S = 4.0,
    coordinates = Coordinates(lat = 33.4480, lon = -112.0740),
    color = "#EF0000",
    state = "Arizona"
)

team_PIT = Team(
    name = "Pittsburgh Riverhounds SC",
    venue = "Highmark Stadium",
    L = 1.0,
    S = 4.0,
    coordinates = Coordinates(lat = 40.4397, lon =-80.0094),
    color = "#FFC72C",
    state = "Pennsylvania"
)

team_TB = Team(
    name = "Tampa Bay Rowdies",
    venue = "Al Lang Stadium",
    L = 1.0,
    S = 4.0,
    coordinates = Coordinates(lat = 27.77062, lon = -82.63730),
    color = "#4A8B2C",
    state = "Florida"
)

team_OAK = Team(
    name = "Oakland Roots SC",
    venue = "Oakland Coliseum",
    L = 1.0,
    S = 4.0,
    coordinates = Coordinates(lat = 37.7529, lon = -122.2005),
    color = "#000000",
    state = "California"
)

leagues_expanded = copy_with_just_league(leagues, league_name)
counties_geojson_expanded = load_counties_geojson()

add_team(co_data_frame, leagues_expanded, league_name, [team_IND, team_SAC , team_PHO, team_DET, team_SA, team_PIT, team_TB, team_OAK])

reset_county_styles(counties_geojson_expanded)
heatmap_counties(leagues_expanded, counties_geojson_updated, league_name) 

render_map(leagues_expanded, league_name, counties_geojson_updated, co_data_frame)

In [ ]:
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.set_option('display.max_rows', 5000)

def league_teams_state_sums(league_data):
    league_dfs = league_data["dataframes"]
    num_teams = len(league_data["json"]["teams"])
    return league_dfs[league_dfs["share"].apply(lambda x: x > 1/num_teams)]\
        [['team_name', 'state', 'county', 'share_population']]\
        .groupby(['team_name', 'state', 'county'], as_index=False)\
        .sum()

def show_pre_post_merged_results(league_before, league_after):
    pre_sums = league_teams_state_sums(league_before)
    post_sums = league_teams_state_sums(league_after)
    merged = pd.merge(pre_sums, post_sums, on=['team_name','state','county'], how='right')
    merged = merged[merged["share_population_y"] > 0]
    merged["share_population_x"].fillna(0, inplace = True) 
    merged["share_population_delta"] = merged["share_population_y"] - merged["share_population_x"]
    print(merged[["team_name", "state", "county", "share_population_delta"]].to_string(index=False))
    print(f"Sums\t: {merged["share_population_x"].sum()}\t{merged["share_population_y"].sum()}")
    print(f"Sum Delta = {merged["share_population_y"].sum() - merged["share_population_x"].sum()}") 


show_pre_post_merged_results(
    leagues[league_name],
    leagues_expanded[league_name], 
)